apt install postgresql

![Challenges of Postgres](./images/ChallengesOfPostgres.png)

Store list of largest cities: https://en.wikipedia.org/wiki/List_of_largest_cities  
Focus only on 4 columns: City, Country, Population, Area

![DatabaseDesignProcess](./images/DatabaseDesignProcess.png)

$ sudo -u postgres psql

Inside Postgres(#):  
CREATE DATABASE db;  
CREATE USER debayan WITH PASSWORD '12345';  
exit

$ psql -h localhost -p 5432 -U debayan db 

DataTypes in Postgres - https://www.postgresql.org/docs/current/datatype.html

CREATE TABLE cities(  
    name VARCHAR(50),  
    country VARCHAR(50),  
    population INTEGER,  
    area INTEGER  
);  

DB: \l  
Connect to a DB: \c db  
Tables: \d  
SELECT * FROM cities;  

### INSERT INTO TABLE

INSERT INTO cities(name, country, population, area)  
VALUES ('Tokyo', 'Japan', 38505000, 8223);  
/* name, population, country, area - can be changed posution and we need to put the data accordingly in VALUES*/

### Insert multiple rows in one go:  
INSERT INTO cities(name, country, population, area)  
VALUES 
    ('Delhi', 'India', 28125000, 2240),  
    ('Shanghai', 'China', 22125000, 4015),  
    ('Sao Paulo', 'Brazil', 20935000, 3043);   

